In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from datetime import timedelta
import joblib
from datetime import date
import ruptures as rpt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
from scipy.stats import pearsonr
import os
from pathlib import Path
from sklearn.model_selection import RepeatedKFold
from datetime import datetime, timedelta

In [ ]:
CWD  = os.path.dirname(os.path.realpath("./deweathering_co.ipynb"))
DIR = Path(CWD).parent
DATA_SRC = DIR / ‘Data’


In [ ]:
co_data=pd.read_csv(DATA_SRC/ ‘CO_2018_2025.csv',index_col=0)

In [ ]:
co_data=co_data.dropna()

In [ ]:
seed = np.random.RandomState(0)

In [ ]:
columns = ['t2m','msl','blh','ssr','mtpr','tcc','wd','ws','rh','hour','day_julian','weekday','date_unix']

In [ ]:
delhi_predictors, delhi_CO = co_data[columns], co_data['CO']

In [ ]:
delhi_predictors_train, delhi_predictors_test, delhi_CO_train, delhi_CO_test = train_test_split(delhi_predictors, delhi_CO, test_size=0.3,random_state=seed)

In [ ]:
gbdt= HistGradientBoostingRegressor(random_state=seed, n_iter_no_change=5, early_stopping=True)
pipeline = Pipeline(steps=[ ("gbdt", gbdt)])
params = {
    "gbdt__max_iter": np.arange(100,500,25),
    "gbdt__learning_rate": np.arange(0.01,0.1,0.03),
    "gbdt__max_depth": np.arange(2,7,1),
    "gbdt__min_samples_leaf": np.arange(1, 20, 1),
}
cv_gbdt = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
search = RandomizedSearchCV(pipeline, param_distributions=params, n_jobs=-1, cv=cv_gbdt, random_state=seed, n_iter = 500)
search.fit(delhi_predictors_train, delhi_CO_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.873):
{'gbdt__min_samples_leaf': 15, 'gbdt__max_iter': 425, 'gbdt__max_depth': 6, 'gbdt__learning_rate': 0.09999999999999999}


In [ ]:
params = {
    "gbdt__learning_rate": [0.099],
    "gbdt__max_depth": np.arange(4, 8, 1),
    "gbdt__min_samples_leaf": np.arange(12, 18, 1),
    "gbdt__max_iter": np.arange(375, 475, 25),
}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid=params, n_jobs=-1, cv=cv_gbdt)
grid_search.fit(delhi_predictors_train, delhi_CO_train)
print(grid_search.best_params_)

{'gbdt__learning_rate': 0.099, 'gbdt__max_depth': 7, 'gbdt__max_iter': 450, 'gbdt__min_samples_leaf': 15}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
joblib.dump(best_model, 'best_model_delhi_2025CO.joblib')

['best_model_delhi_2025CO.joblib']

In [ ]:
print(f'R2 score (train): {best_model.score(delhi_predictors_train, delhi_CO_train):.4f}')
print(f'R2 score (test): {best_model.score(delhi_predictors_test, delhi_CO_test):.4f}')

R2 score (train): 0.9320
R2 score (test): 0.8834


In [ ]:
CO_prediction_test = best_model.predict(delhi_predictors_test)

In [ ]:
predictions_df = pd.DataFrame({'Predicted_CO': CO_prediction_test})


In [ ]:
predictions_df['Actual_CO'] = co_data.loc[delhi_predictors_test.index, 'CO'].values
predictions_df['date'] = co_data.loc[delhi_predictors_test.index, 'date'].values

In [ ]:
predictions_df.to_csv('delhi_CO_test_predictions_2025.csv', index=False)

In [ ]:
delhi_predictors_train.to_csv("delhi_CO_predictors_train_2025.csv", index=False)
delhi_CO_train.to_csv("delhi_CO_train_2025.csv", index=False)
delhi_predictors_test.to_csv("delhi_CO_predictors_test_2025.csv", index=False)
delhi_CO_test.to_csv("delhi_CO_test_2025.csv", index=False)

In [ ]:
mae = mean_absolute_error(best_model.predict(delhi_predictors_test), delhi_CO_test)
rmse = mean_squared_error(best_model.predict(delhi_predictors_test), delhi_CO_test, squared=False)
mbe = np.mean(best_model.predict(delhi_predictors_test) - delhi_CO_test)
r = pearsonr(best_model.predict(delhi_predictors_test), delhi_CO_test)
mape = np.mean(np.abs((delhi_CO_test - best_model.predict(delhi_predictors_test)) / delhi_CO_test)) * 100
print(mae, rmse, mbe, r, mape)

167.16841315131205 251.56342419264118 0.9213703543208134 PearsonRResult(statistic=0.9400831159482266, pvalue=0.0) 12.307726850594634


/home/ceew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
nrows_data = len(co_data.index)

53327

In [ ]:
normalised_data = pd.DataFrame()
for i in range(2000):
    met_shuffled = delhi_predictors.drop(axis=1, columns=['date_unix','day_julian','weekday','hour']).sample(n=nrows_data).reset_index(drop=True);
    met_shuffled[['date_unix','day_julian','weekday','hour']] = delhi_predictors[['date_unix','day_julian','weekday','hour']];
    prediction_data = best_model.predict(met_shuffled[columns]);
    normalised_data[i] = prediction_data;
    if (i%100 == 0):
        print(i)
    warnings.filterwarnings('ignore')

In [ ]:
mean_df = normalised_data.mean(axis=1).reset_index()
mean_df.columns = ['Index', 'Mean_Value']

In [ ]:
co_data.reset_index(drop=True, inplace=True)

In [ ]:
co_data['deweathered_CO'] = mean_df['Mean_Value']
co_data['date'] = pd.to_datetime(co_data['date'])
co_data['met_CO'] = co_data['CO'] - co_data['deweathered_CO']

In [ ]:
co_data.to_csv('DeweatheredCO_2018_2025.csv')